# Anomaly detection

The dataset is public and references to its license could be found in the README.md in /data subdirectory of the repo.

In [1]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors

In [33]:
from mxnet import autograd, np, npx, gluon, optimizer, init
from mxnet.gluon import nn

In [3]:
# This is the entry point of our spark app
spark = SparkSession \
        .builder \
        .appName("2-class classification") \
        .getOrCreate()

## Where to download the dataset

The dataset we use can downloaded from https://raw.githubusercontent.com/alessio-proietti/dp-sgd-notebook/main/data/bank-additional-full-new-label.csv. 

It's shipped with the repo itself though.

In [4]:
df = spark.createDataFrame(pd.read_csv("data/bank-additional-full-new-label.csv"))

In [5]:
print(df.columns)
df.drop('duration')
df.describe(['age', 'campaign', 'pdays', 'previous']).show()
df.describe(['emp_var_rate', 'cons_price_idx', 'con_conf_idx', 'euribor3m', "nr_employed"]).show()
print('yes:', df.filter(df['y'] == 'yes').count())
print('no:', df.filter(df['y'] == 'no').count())

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'con_conf_idx', 'euribor3m', 'nr_employed', 'y']
+-------+------------------+-----------------+------------------+-------------------+
|summary|               age|         campaign|             pdays|           previous|
+-------+------------------+-----------------+------------------+-------------------+
|  count|             41188|            41188|             41188|              41188|
|   mean| 40.02406040594348|2.567592502670681| 962.4754540157328|0.17296299893172767|
| stddev|10.421249980934048|2.770013542902328|186.91090734474213| 0.4949010798392895|
|    min|                17|                1|                 0|                  0|
|    max|                98|               56|               999|                  7|
+-------+------------------+-----------------+------------------+---

In [6]:
# Here we set the stages for data processing

numericCols = [field for (field, dataType) in df.dtypes if ( dataType != "string" )]
categoricalCols = [field for (field, dataType) in df.dtypes if (dataType == "string" and field != "y")]

indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=oheOutputCols)

assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [7]:
stringIndexerLabel = StringIndexer(inputCol="y", outputCol="label", handleInvalid="skip")
labelModel = stringIndexerLabel.fit(df)
df = labelModel.transform(df)

In [8]:
train, validation = df.randomSplit([.75, .25], 24)

pipeline = Pipeline(stages=[stringIndexer, oheEncoder, vecAssembler])

pipelineModel = pipeline.fit(train)
train_df = pipelineModel.transform(train).select("label","features")

pipelineModel = pipeline.fit(validation)
val_df = pipelineModel.transform(validation).select("label","features")

In [ ]:
#numpy array costruct

In [ ]:
#Hyperparameters
batch_size = 500
num_epochs = 100

In [54]:
npx.set_np()

@optimizer.Optimizer.register
class PrivateSGD(optimizer.Optimizer):
    
    def __init__(self, learning_rate=0.1, momentum=0.0, use_fused_step=False, **kwargs):
        super(PrivateSGD, self).__init__(learning_rate=learning_rate, **kwargs)
        self.momentum = momentum
        
    def create_state(self, index, weight):
        return None
    
    def step(self, indices, weights, grads, states):
        for index, weight, grad, state in zip(indices, weights, grads, states):
            self._update_count(index)
            lr = self._get_lr(index)
            wd = self._get_wd(index)

            # preprocess grad
            grad *= self.rescale_grad
            if self.clip_gradient is not None:
                grad = clip(grad, -self.clip_gradient, self.clip_gradient)
            grad += wd * weight

            # update mom
            mom = state
            if mom is not None:
                mom[:] *= self.momentum
                mom[:] -= lr * ( grad + sigma*np.random_normal(shape = grad.shape) )
            else:
                mom = -lr * ( grad + sigma*np.random_normal(shape = grad.shape) )

            # update weight
            weight[:] += mom
       

In [55]:
trainer = gluon.Trainer(net.collect_params(), 'PrivateSGD', {'learning_rate': 0.1})

In [49]:
net = nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

In [9]:
#spark.stop()